In [84]:
import itertools
import pandas as pd
import numpy as np
import random

"""

To help code our matrix, we will give names to all our states.
I choose to describe the position of each disc using Cartesian coordinates,
where the x-coordinate represents the disc (1,2, or 3),
and the y-coordinate represents the height (1,2,3 or 3).
We can nest these coordinates into a tuple,
where the first position in the tuple will note the coordinates for the largest disc,
the second position the middle-radius disc, and the third coordinate the smallest disc. 

For example:

((2,1),(2,2), (2,3)) represents the starting state
((2,1),(2,2), (1,1)) represents a potential second state, as shown in the first picture above
((1,1),(1,2), (1,3)) and ((3,1),(3,2), (3,3)) represent the winning states (“absorbing” states in Markov parlance), which end the game.


With this framework, we need to first define which states are valid. 
We create a series of functions that check a given state for various aspects of validity.
We create a final function that checks all the these functions and returns True (valid) or False (invalid).

"""

# Remove all duplicates
def filter_duplicate_coordinates(state):
    return True if len(state) == len(set(state)) else False

"""
If the largest disc ("a") shares an x-coordinate with the medium disc ("b"), 
# "a" must be below "b"
# E.g., if X_a == Y_b -> Y_a < Y_b
# We should generalize this function, but we'll code it manually for the three cases for now.
""" 

# If X_a == X_b -> Y_a < Y_b
def filter_Ya_Yb(state):
    if state[0][0] == state[1][0]:
        return int(state[0][1]) < int(state[1][1])
    return True

# If X_b == X_c -> Y_b < Y_c
def filter_Yb_Yc(state):
    if state[1][0] == state[2][0]:
        return int(state[1][1]) < int(state[2][1])
    return True

# If X_a == X_b -> Y_a < Y_c
def filter_Ya_Yc(state):
    if state[0][0] == state[2][0]:
        return int(state[0][1]) < int(state[2][1])
    return True

"""
Y coordinate can only be 3 if Y_a == 1 and Y_b == 2, and X_a == X_b == X_c
"""

def filter_valid_y_coord_1(state):
    return False if state[0][1] in ('2', '3') or state[1][1] == '3' else True


def filter_valid_y_coord_2(state):
    if state[2][1] == '3':
        return all(
            (state[0][0] == state[1][0], state[0][0] == state[2][0], state[0][1] == '1',
             state[1][1] == '2'))
    return True

"""
Y coordinate can only be 2 if Y_a == 1 or Y_b == 1, and (X_a == X_b or X_a == X_c or X_b == X_c)
"""

def filter_valid_y_coord_3(state):
    if state[2][1] == '2':
        return any((state[0][0] == state[2][0], state[1][0] == state[2][0]))
    return True


def filter_valid_y_coord_4(state):
    if state[1][1] == '2':
        return state[0][0] == state[1][0]
    return True
        
# Filter all
def filter_all(state):
    return all((
        filter_duplicate_coordinates(state),
        filter_Yb_Yc(state),
        filter_Ya_Yc(state),
        filter_valid_y_coord_1(state),
        filter_valid_y_coord_2(state),
        filter_valid_y_coord_3(state),
        filter_valid_y_coord_4(state)
    ))


"""

This function checks whether state1 can validily transition into state2.
Returns True if the transition is valid and False if the transition is invalid.

"""

def valid_transition(state1, state2):
    
    # Cannot transition to same state
    if state1 == state2:
        return False

    # Confirm only one piece moved
    if not any((
            (state1[0] == state2[0] and state1[1] == state2[1]),
            (state1[1] == state2[1] and state1[2] == state2[2]),
            (state1[0] == state2[0] and state1[2] == state2[2]),
    )):
        return False
    
    moved_piece = set(state1) - set(state2)

    # A covered piece cannot be moved
    # Get covered pieces and make sure they weren't the moved piece
    covered_pieces = []
    if state1[0][0] == state1[1][0] or state1[0][0] == state1[2][0]:
        covered_pieces.append(state1[0])
    if state1[1][0] == state1[2][0]:
        covered_pieces.append(state1[1])
    if list(moved_piece)[0] in covered_pieces:
        return False

    return True


# Setup by creating potential state list
coords = ['1', '2', '3']
piece_states = list(itertools.product(coords, coords))
states_unfiltered = list(itertools.product(piece_states, repeat=3))

# Filter to valid states only
states = list(filter(filter_all, states_unfiltered))

# Create a dictionary that finds, for each state, the valid states it can transition into
transitions = {}
for state1 in states:
    transitions[state1] = set()
    for state2 in states:
        if valid_transition(state1, state2) and state1 != state2:
            transitions[state1].add(state2)

# Override transitions for ((1, 1), (1, 2), (1, 3)) and ((3, 1), (3, 2), (3, 3)) as the absorbing states
# These states can only transition to themselves
transitions[(('1', '1'), ('1', '2'), ('1', '3'))] = {(('1', '1'), ('1', '2'), ('1', '3'))}
transitions[(('3', '1'), ('3', '2'), ('3', '3'))] = {(('3', '1'), ('3', '2'), ('3', '3'))}

# Compute transition probabilities and store in a list
transition_p = []
for state in transitions:
    p = 1 / len(transitions[state])
    for state2 in transitions[state]:
        transition_p.append([state, state2, p])

# Convert list to transition matrix format
df = pd.DataFrame(transition_p, columns=['state', 'state_new', 'p'])
df_pivot = df.pivot_table(index='state', columns='state_new', values='p', fill_value=0, dropna=False)
matrix = df_pivot.to_numpy()
matrix = np.roll(matrix, -1, axis=[0, 1]) # Move absorbing state last

# Time to absorption calculation
number_of_absorbing_states = 2
I = np.eye(len(matrix) - number_of_absorbing_states)
Q = matrix[:number_of_absorbing_states*-1,:number_of_absorbing_states*-1]
N = np.linalg.inv(I - Q)
o = np.ones(Q.shape[0])
tta = np.dot(N, o)

print('Exact solution: ' + str(np.max(tta)))

Exact solution: 70.77777777777752


In [83]:
# Monte Carlo check
def monte_carlo_run():
    state = tuple((('2', '1'), ('2', '2'), ('2', '3'))) # Starting state
    moves = 0
    while state not in (states[0],states[-1]): # Victory states
        
        # New state is randomly selected from valid transition states
        state = random.choice(tuple(filter(lambda s: valid_transition(state, s),states)))
        moves += 1
    
    return moves

N = 100000
total_moves = 0
for _ in range(N):
    total_moves += monte_carlo_run()
print('Monte Carlo check: ' + str(total_moves/N))

Monte Carlo check: 70.8608
